<a href="https://colab.research.google.com/github/mzaoualim/Business_Proposals/blob/main/Hierarchical_sales_forecasting/Steps_HTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Scope

# Modules

In [3]:
!pip install -q hierarchicalforecast
!pip install -q -U numba statsforecast datasetsforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.2/508.2 kB 34.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
from datasetsforecast.hierarchical import HierarchicalData, HierarchicalInfo
from hierarchicalforecast.utils import aggregate, HierarchicalPlot
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Data

## load

In [5]:
# data retrieval

from google.colab import files
files.upload()

Saving HistoricalProductDemand.csv to HistoricalProductDemand.csv


In [6]:
# read csv file

data = pd.read_csv('/content/HistoricalProductDemand.csv')

In [7]:
data.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


## Format


In [10]:
# missing data points

data.isna().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64

In [11]:
data.dropna(inplace=True)

In [12]:
# fixing the data types

data.dtypes

Product_Code        object
Warehouse           object
Product_Category    object
Date                object
Order_Demand        object
dtype: object

In [13]:
# convert Order_Demand to integers and set rows with non-numeric values to NaN:
data['Order_Demand'] = pd.to_numeric(data['Order_Demand'], errors='coerce', downcast='float')

# convert Date to datetime dtypes
data['Date'] = pd.to_datetime(data['Date'])

In [14]:
data.dtypes

Product_Code                object
Warehouse                   object
Product_Category            object
Date                datetime64[ns]
Order_Demand               float32
dtype: object

In [15]:
# NaN data points

data.isna().sum()

Product_Code           0
Warehouse              0
Product_Category       0
Date                   0
Order_Demand        5899
dtype: int64

In [16]:
data.dropna(inplace=True)

## Quick EDA

In [29]:
# overview of hierarchical strutcure

fig = px.sunburst(
    data,
    path=['Warehouse', 'Product_Category', 'Product_Code'],
    values='Order_Demand'
                  )

fig.update_layout(
    title_font_color='blue',
    title={
          'text': 'Exploring Sales Structure',
          'y':.95,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top'
          }
    )

fig.update_traces(textinfo="label+percent entry")

fig.show()

Sales are dominated by:


1.   In Warehouse level by Whse_J
2.   In Product Category level by Category_19

By clicking on the sunburst chart, we get an interactive plot of different level/hierarchy of sales.



In [34]:
# lets have a quick presentation of cyclical/trending sales

data.groupby('Date')['Order_Demand'].sum()

Date
2011-01-08         2.0
2011-05-31       108.0
2011-06-24     92000.0
2011-09-02      1250.0
2011-09-27        28.0
                ...   
2017-01-03      2400.0
2017-01-04     29250.0
2017-01-05     83929.0
2017-01-06    175140.0
2017-01-09      4178.0
Name: Order_Demand, Length: 1727, dtype: float32

In [35]:
# overview of cycles and trends of Order_Demande

fig = px.line(data.groupby('Date')['Order_Demand'].sum())
fig.show()

In [50]:
# Monthly total sales

fig = px.line(data.set_index('Date').groupby(pd.Grouper(freq='M'))['Order_Demand'].sum())

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.update_layout(
    title_font_color='blue',
    title={
          'text': 'Exploring Total Sales Cycles and Trend',
          'y':.95,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top'
          }
    )

fig.show()

Total monthy sales seems to be stable over the span of 6 years.

In [62]:
data['Warehouse'].unique()

array(['Whse_J', 'Whse_S', 'Whse_C', 'Whse_A'], dtype=object)

In [68]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=len(data['Warehouse'].unique()), cols=1)
for a in data['Warehouse'].unique():
  fig = px.line(data.loc[data['Warehouse'] == a].set_index('Date').groupby(pd.Grouper(freq='M'))['Order_Demand'].sum())

  fig.update_xaxes(
      dtick="M1",
      tickformat="%b\n%Y",
      ticklabelmode="period")

  fig.update_layout(
      title_font_color='blue',
      title={
            'text': 'Exploring Warehouse ' + a + ' Total Sales Cycles and Trend',
            'y':.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
            }
      )

  fig.show()

In [57]:
data.loc[data['Warehouse'] == 'Whse_J']

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012-07-27,100.0
1,Product_0979,Whse_J,Category_028,2012-01-19,500.0
2,Product_0979,Whse_J,Category_028,2012-02-03,500.0
3,Product_0979,Whse_J,Category_028,2012-02-09,500.0
4,Product_0979,Whse_J,Category_028,2012-03-02,500.0
...,...,...,...,...,...
1048570,Product_1791,Whse_J,Category_006,2016-04-27,1000.0
1048571,Product_1974,Whse_J,Category_006,2016-04-27,1.0
1048572,Product_1787,Whse_J,Category_006,2016-04-28,2500.0
1048573,Product_0901,Whse_J,Category_023,2016-10-07,50.0


In [55]:
data[['Date','Warehouse', 'Order_Demand']].set_index('Date').groupby(pd.Grouper(freq='M'))['Order_Demand'].sum()

Date
2011-01-31           2.0
2011-02-28           0.0
2011-03-31           0.0
2011-04-30           0.0
2011-05-31         108.0
                 ...    
2016-09-30    77654480.0
2016-10-31    83927472.0
2016-11-30    89989240.0
2016-12-31    80402120.0
2017-01-31      294967.0
Freq: M, Name: Order_Demand, Length: 73, dtype: float32

## Preprocessing

In [ ]:
data.rename(columns={'Date':'ds', 'Order_Demand': 'y'},inplace=True)

In [ ]:
# data structure
'''
Warehouse = ['Whse_J', 'Whse_S', 'Whse_C', 'Whse_A']
    |
    |----Product_Category = [
      'Category_028', 'Category_006', 'Category_001', 'Category_019',
      'Category_005', 'Category_030', 'Category_032', 'Category_011',
      'Category_033', 'Category_007', 'Category_015', 'Category_021',
      'Category_024', 'Category_026', 'Category_023', 'Category_022',
      'Category_009', 'Category_003', 'Category_004', 'Category_018',
      'Category_020', 'Category_013', 'Category_008', 'Category_017',
      'Category_031', 'Category_025', 'Category_010', 'Category_029',
      'Category_012', 'Category_002', 'Category_014', 'Category_027',
      'Category_016'
      ]
      |
      |----------- Product_Code = [
        'Product_0993', 'Product_0979', 'Product_1159', ...,
        'Product_0237', 'Product_0644', 'Product_0853'
        ](2160)
'''

In [ ]:
#example to set S matrix

# Assuming Y_df is your DataFrame containing the time series data
# with columns ['Country', 'State', 'Region', ... , 'ds', 'y']
# and 'hiers' is a list of lists specifying the hierarchy levels
Y_df = data.copy()
specs = [
    ['Warehouse'],
    ['Warehouse', 'Product_Category'],
    ['Warehouse', 'Product_Category', 'Product_Code'],
    # Add more hierarchy levels as needed
]

# Use the aggregate function to create the aggregation matrix S
Y_df, S_df, tags = aggregate(data, specs)

# Y_df_agg is the aggregated DataFrame,
# S_df is the aggregation matrix S,
# and 'tags' helps in understanding the aggregation levels

In [ ]:
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
Y_df.reset_index(inplace=True)
Y_df.head()

,unique_id,ds,y
0,Whse_A,2011-01-08,2.0
1,Whse_A,2011-05-31,108.0
2,Whse_A,2011-11-18,4001.0
3,Whse_A,2011-11-21,110.0
4,Whse_A,2011-11-22,3700.0


In [ ]:
S_df.head()

,Whse_A/Category_001/Product_1751,Whse_A/Category_003/Product_1180,Whse_A/Category_003/Product_1719,Whse_A/Category_003/Product_1724,Whse_A/Category_003/Product_1725,Whse_A/Category_003/Product_1726,Whse_A/Category_003/Product_1727,Whse_A/Category_005/Product_0001,Whse_A/Category_005/Product_0020,Whse_A/Category_005/Product_0031,...,Whse_S/Category_032/Product_0586,Whse_S/Category_032/Product_0587,Whse_S/Category_032/Product_0588,Whse_S/Category_032/Product_0589,Whse_S/Category_032/Product_0591,Whse_S/Category_032/Product_0592,Whse_S/Category_032/Product_0593,Whse_S/Category_032/Product_0595,Whse_S/Category_032/Product_1052,Whse_S/Category_032/Product_1101
Whse_A,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Whse_C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Whse_J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Whse_S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Whse_A/Category_001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Y_test_df = Y_df.groupby('unique_id').tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

In [ ]:
Y_train_df.groupby('unique_id').size()

unique_id
Whse_A                              1567
Whse_A/Category_001                   88
Whse_A/Category_001/Product_1751      88
Whse_A/Category_003                  945
Whse_A/Category_003/Product_1180     179
                                    ... 
Whse_S/Category_032/Product_0592      49
Whse_S/Category_032/Product_0593     100
Whse_S/Category_032/Product_0595      39
Whse_S/Category_032/Product_1052      43
Whse_S/Category_032/Product_1101      92
Length: 2909, dtype: int64

# Modeling

In [ ]:
from statsforecast.models import AutoARIMA
from statsforecast.core import StatsForecast

In [ ]:
fcst = StatsForecast(df=Y_train_df,
                     models=[AutoARIMA(season_length=7)],
                     freq='D', n_jobs=-1)

Y_hat_df = fcst.forecast(h=7, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:551: FutureWarning: Passing unique_id as the index is deprecated. Please provide it as a column instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/arima.py:1536: RuntimeWarning: divide by zero encountered in scalar 

KeyboardInterrupt: 

In [ ]:
import cloudpickle
with open('mlf.pkl', 'wb') as f:
    cloudpickle.dump(fcst, f)

# Hierarchical reconciliation

In [ ]:
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df, tags=tags)

# Results & Insights

In [ ]:
from hierarchicalforecast.evaluation import HierarchicalEvaluation

In [ ]:
def rmse(y, y_hat):
    return np.mean(np.sqrt(np.mean((y-y_hat)**2, axis=1)))

def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['Purpose'] = tags['Country/Purpose']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']
eval_tags['Bottom'] = tags['Country/State/Region/Purpose']
eval_tags['All'] = np.concatenate(list(tags.values()))

evaluator = HierarchicalEvaluation(evaluators=[rmse, mase])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=eval_tags, Y_df=Y_train_df
)
evaluation = evaluation.drop('Overall')
evaluation.columns = ['Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']
evaluation = evaluation.applymap('{:.2f}'.format)


In [ ]:
# eval and crossval

In [ ]:
# plots and viz

# Ressources

[Data](https://www.kaggle.com/datasets/felixzhao/productdemandforecasting?select=Historical+Product+Demand.csv)

[HIERARCHICAL FORECASTING: RETAIL INVENTORY PLANNING](https://josephine-amponsah.medium.com/forecasting-demand-of-retail-inventory-hierarchical-approach-3ec9c42a65e8)

[scikit-hts showcase: M5 kaggle competition](https://github.com/carlomazzaferro/scikit-hts-examples/blob/master/notebooks/M5.ipynb)